In [1]:
from pyspark.sql import SparkSession


from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD, LogisticRegressionWithLBFGS

In [2]:
#pyspark init
builder = SparkSession.builder\
            .appName('creditcard_mllib')\
            .config("spark.driver.memory", "4g")\
            .config("spark.executor.memory", "4g")
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/12 21:25:14 WARN Utils: Your hostname, minhnhat resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/12 21:25:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/12 21:25:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Read input file
raw_df = spark.read.csv('creditcard.csv', header=True, inferSchema=True)

## Preprocessing

In [4]:
#Convert to rdd
feature_cols = [col for col in raw_df.columns if col != 'Class']

data_rdd = raw_df.rdd.map(lambda row: LabeledPoint(
    float(row['Class']),
    [float(row[col]) for col in feature_cols]
))

In [5]:
data_rdd.take(5)

[LabeledPoint(0.0, [0.0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62]),
 LabeledPoint(0.0, [0.0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.183361270123994,-0.145783041325259,-0.0690831352230203,-0.225775248033138,-0.638671952771851,0.101288021253234,-

In [6]:
#Split data into training and testing sets

train_size = 0.8
train_rdd, test_rdd = data_rdd.randomSplit([train_size, 1 - train_size], seed=24)

## Fit the model

In [7]:
iterations = 100

model = LogisticRegressionWithLBFGS().train(train_rdd, iterations=iterations, intercept=True)

25/04/12 21:25:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [8]:
# Display model coefficients and intercept
print("Coefficients: ", model.weights)
print("Intercept: ", model.intercept)

Coefficients:  [-4.1734877493517316e-06,0.07662392514300612,0.042576005566222104,-0.0025974604475015716,0.728117805028605,0.1548178159566063,-0.10603229688637944,-0.06652750049964885,-0.17021858996062536,-0.21069198107111778,-0.8224884824164258,-0.08346773549046786,0.07709858202158607,-0.3347891111209767,-0.5247270553377439,-0.05392097012398126,-0.20783234800975595,-0.018626752648998664,0.04524432021030166,0.11351828994636996,-0.4274756204245057,0.35243966079249345,0.5662010105390634,-0.10486905438093105,0.15177209832970556,6.03011803522605e-05,0.03650360784859915,-0.7694951651918046,-0.2443587814146048,0.0008685648478534997]
Intercept:  -8.311975875206434


## Evaluate the model

In [9]:
def compute_accuracy(predictions_rdd):
    correct = predictions_rdd.filter(lambda x: x[0] == x[1]).count()
    total = predictions_rdd.count()
    return correct / total if total > 0 else 0.0

def compute_precision(predictions_rdd, label):
    true_positive = predictions_rdd.filter(lambda x: x[0] == label and x[1] == label).count()
    predicted_positive = predictions_rdd.filter(lambda x: x[0] == label).count()
    return true_positive / predicted_positive if predicted_positive > 0 else 0.0

def compute_recall(predictions_rdd, label):
    true_positive = predictions_rdd.filter(lambda x: x[0] == label and x[1] == label).count()
    actual_positive = predictions_rdd.filter(lambda x: x[1] == label).count()
    return true_positive / actual_positive if actual_positive > 0 else 0.0

def compute_auc(predictions_rdd):
    n_pos = predictions_rdd.filter(lambda x: x[1] == 1).count()
    n_neg = predictions_rdd.filter(lambda x: x[1] == 0).count()
    if n_pos == 0 or n_neg == 0:
        return 0.0

    data = predictions_rdd.collect()
    data.sort(key=lambda x: x[0])
    
    rank_sum = 0.0
    rank = 1
    i = 0
    while i < len(data):
        j = i

        while j < len(data) and data[j][0] == data[i][0]:
            j += 1
        group_size = j - i
        avg_rank = (2 * rank + group_size - 1) / 2.0
        for k in range(i, j):
            if data[k][1] == 1:
                rank_sum += avg_rank
        rank += group_size
        i = j

    return (rank_sum - (n_pos * (n_pos + 1) / 2.0)) / (n_pos * n_neg)

In [10]:
#Evaluate the model on training data
predictions_train = train_rdd.map(lambda p: (model.predict(p.features), p.label))

print("Train Accuracy:", compute_accuracy(predictions_train))
print("Train Precision: ", [compute_precision(predictions_train, label) for label in [0, 1]])
print("Train Recall: ", [compute_recall(predictions_train, label) for label in [0, 1]])
print("Train AUC: ", compute_auc(predictions_train))

Train Accuracy: 0.9991568000562133


Train Precision:  [0.999314083708169, 0.8671586715867159]


Train Recall:  [0.9998416280635072, 0.6010230179028133]


Train AUC:  0.8004323229831602


In [11]:
#Evaluate the model on test data
predictions_test = test_rdd.map(lambda p: (model.predict(p.features), p.label))

print("Test Accuracy:", compute_accuracy(predictions_test))
print("Test Precision: ", [compute_precision(predictions_test, label) for label in [0, 1]])
print("Test Recall: ", [compute_recall(predictions_test, label) for label in [0, 1]])
print("Test AUC: ", compute_auc(predictions_test))

Test Accuracy: 0.9992469747648985


Test Precision:  [0.9993337424388533, 0.9264705882352942]


Test Recall:  [0.9999122837795165, 0.6237623762376238]


Test AUC:  0.8118373300085702


In [12]:
#Stop the Spark session
spark.stop()